## Import Packages
We need to install transformers and datasets. soundfile is used to load audio files and jiwer is used to evaluate the finetuned model using word errot rate

In [ ]:
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

To upload our training checkpoints directly to huggingface, we have to store the huggingface authentication key.

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Install the GIT LFS in order to upload the model checkpoints

In [5]:
!apt install git-lfs

# Prepare Data, Tokenizer, Feature Extractor 

### Create Wav2Vec2CTCTokenizer 

In [7]:
# Load the dataset
from datasets import load_dataset, load_metric

# You can pass the streaming option to load_dataset to stream the data from the source instead of downloading and caching it
luganda = load_dataset("mozilla-foundation/common_voice_16_1", "lg")

print(luganda)

/Users/denis/miniforge3/envs/idl/lib/python3.8/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_1
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Reading metadata...: 71069it [00:00, 258823.31it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

Reading metadata...: 13331it [00:00, 152638.60it/s]


...

Reading metadata...: 13358it [00:00, 255269.24it/s]


Generating other split: 0 examples [00:00, ? examples/s]

Reading metadata...: 36998it [00:00, 277481.30it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]

Reading metadata...: 39170it [00:00, 268377.68it/s]


In [10]:
luganda

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 71069
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 13331
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 13358
    })
    other: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 36998
    })
    invalidated: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_

In [11]:
# Remove the unnecessary columns from the dataset
luganda = luganda.remove_columns(["client_id", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "variant"])

### Display some of the rows in the dataset

In [13]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(luganda["train"].remove_columns(["path", "audio"]))


,sentence
0,Ab’e Buddu bakola nnyo.
1,Njagala kugenda mu kibuga.
2,Ebisolo ebingi awamu biyitibwa eggana.
3,Nga tonnaba kusalawo sooka olowooze ku kintu ekiddako.
4,Abantu bafuna empeereza y'obujjanjabi embi.
5,Kati nsuubira Mukuumaddamula okwogera ku nsonga y'emirimu egigootaanye olwa Corona
6,Omugoba w'ekidduka ateekeddwa okugoberera amateeka g'enguudo okusobola okwewala obubenje.
7,Akasawo kano kalabika bulungi.
8,Buvunaanyizibwa bwa gavumenti okulaba ng'abaana bonna basoma.
9,Obulwadde bwa Anthrax buzibu nnyo okutangira.


In [15]:
# Let's normalize the dataset to only lower case letters and ignore any special tokens because without a language model it is difficult to classify such tokens as they do not correspond to a characteristic sound.
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    return batch

luganda = luganda.map(remove_special_characters)

Map:   0%|          | 0/71069 [00:00<?, ? examples/s]

Map:   0%|          | 0/13331 [00:00<?, ? examples/s]

Map:   0%|          | 0/13358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36998 [00:00<?, ? examples/s]

Map:   0%|          | 0/39170 [00:00<?, ? examples/s]

In [17]:
# Display samples from the normalized dataset
show_random_elements(luganda["train"].remove_columns(["path", "audio"]))

,sentence
0,okunaaba kunsumbuwa nnyo bannange
1,yasuulako sikaati n'adduka n'agenda
2,kisobka okuba nga buleetebwawo endwadde ezikwata abakyala nga bali mbuto
3,ekika ky’ekitooke ekiriko enjogera egamba nti “bannabuddu nnakinnyika yabannyika” kiyitibwa nnakinnyika
4,ono yali mukwano gwa long john silver nnyo era nga bwe nkoonye ku linnya lye
5,musigaleewo okutuusa ng'akalulu kammwe bamaze okukabala
6,ssente z'onsaba kumpi zaagala kwenkana z’asaasula ebisale by'essomero
7,yava ewaka nga amaze okuyomba n'abaayo
8,omuvuzi wa loole yaloopye omusango ku poliisi
9,ebigezeso ebyo bisobola okugezebwa omulundi gumu mu kugezesebwa kumu


In CTC chunks of speech are classified into letters. We need to extract all distinct letters in the dataset and builf a vocabulary.   
We need a mapping function that will concatenate all the transcriptions into a long transcription and transforms the strings into a set of characters.

In [21]:
# Let's use the batched = True so that the map function can access all the transcriptions at a go
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = luganda.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=luganda.column_names["train"])

Map:   0%|          | 0/71069 [00:00<?, ? examples/s]

Map:   0%|          | 0/13331 [00:00<?, ? examples/s]

Map:   0%|          | 0/13358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36998 [00:00<?, ? examples/s]

Map:   0%|          | 0/39170 [00:00<?, ? examples/s]

In [22]:
# Create a vocabulary of all letters in the train dataset
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'i': 0,
 'e': 1,
 'é': 2,
 '(': 3,
 'w': 4,
 'ñ': 5,
 'g': 6,
 'c': 7,
 'η': 8,
 'ղ': 9,
 '½': 10,
 'š': 11,
 'a': 12,
 ')': 13,
 '῝': 14,
 '”': 15,
 '‘': 16,
 '—': 17,
 't': 18,
 'd': 19,
 "'": 20,
 'p': 21,
 '“': 22,
 'l': 23,
 'z': 24,
 'u': 25,
 'b': 26,
 'j': 27,
 'f': 28,
 'ó': 29,
 'á': 30,
 'k': 31,
 '/': 32,
 '&': 33,
 '`': 34,
 'q': 35,
 'v': 36,
 'm': 37,
 '‟': 38,
 '–': 39,
 '῎': 40,
 'ŋ': 41,
 'y': 42,
 'h': 43,
 'n': 44,
 'x': 45,
 's': 46,
 '’': 47,
 'o': 48,
 'r': 49,
 '…': 50,
 '`': 51,
 ' ': 52,
 ']': 53}

We need to replace the " " in the dataset with a more visible character. We also need to add the UNKNOWN token so that to deal with characters not encountered in the training dataset. 

In [25]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
# Clean up the vocab_dict
del vocab_dict["é"]
del vocab_dict["("]
del vocab_dict["ñ"]
del vocab_dict["῝"]
del vocab_dict["η"]
del vocab_dict["ղ"]
del vocab_dict["½"]
del vocab_dict["š"]
del vocab_dict["ó"]
del vocab_dict["á"]
del vocab_dict["/"]
del vocab_dict["῎"]
del vocab_dict["ŋ"]
del vocab_dict["…"]
del vocab_dict["]"]
del vocab_dict[")"]
del vocab_dict["&"]

We need to add the pad token that corresponds to CTC's blank token. The blank token is a core component of the CTC algorithm.

In [30]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

In [33]:
# Save the vocabulary to a json file
import json
with open('../../vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [34]:
# Use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("../../vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [35]:
# Push the tokenizer to the hub
repo_name = "luganda_wav2vec2_ctc_tokenizer" 
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/dmusingu/luganda_wav2vec2_ctc_tokenizer/commit/871a447576b60d52b5395aa401f9fba8280555c3', commit_message='Upload tokenizer', commit_description='', oid='871a447576b60d52b5395aa401f9fba8280555c3', pr_url=None, pr_revision=None, pr_num=None)

# Create Wav2Vec Feature Extractor

In [36]:
# Create a feature extractor using Wav2Vec2FeatureExtractor. We shall pass feature size as 1 because we are dealing with raw audio files.
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [37]:
# Wrap the feature extractor and the tokenizer into a Wav2VecProcessor
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Preprocess the dataset

In [41]:
# Dislay an audio sample from the dataset
luganda['train'][10]["audio"]

{'path': '/Users/denis/.cache/huggingface/datasets/downloads/extracted/0a4200d1511403462ed21af0c29a4e8a31c6d5747ce75f8adcaaae6ad13d7250/lg_train_0/common_voice_lg_28085622.mp3',
 'array': array([3.55271368e-15, 2.84217094e-14, 4.61852778e-14, ...,
        4.25824874e-05, 3.32006457e-05, 1.59877036e-05]),
 'sampling_rate': 48000}

In [43]:
# Listen to sample audio from the dataset
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(luganda["train"]))

print(luganda["train"][rand_int]["sentence"])
ipd.Audio(data=np.asarray(luganda["train"][rand_int]["audio"]["array"]), autoplay=True, rate=48000)

In [51]:
rand_int = random.randint(0, len(luganda["train"]))

print("Target text:", luganda["train"][rand_int]["sentence"])
print("Input array shape:", np.asarray(luganda["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", luganda["train"][rand_int]["audio"]["sampling_rate"])

In [54]:
# Convert the sampling frewquency to 16kHz since the model was pretrained on audio sampled at 16kHz
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
# Apply the map function to the dataset
luganda = luganda.map(prepare_dataset, remove_columns=luganda.column_names["train"], num_proc=4)

### Training and Evaluation

In [ ]:
# Set up the trainer
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
# Initialize the data_collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
# Load the metric
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
# Load the pretrained Wav2Vec2 checkpoint. We use the tokenizers pad token id to degine the model's pad token id
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

In [ ]:
# Freeze the feature extractor
model.freeze_feature_extractor()

In [ ]:
# Define the training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=luganda["train"],
    eval_dataset=luganda["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Push the model to hub
trainer.push_to_hub(repo_name)

### Evaluation

processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

In [ ]:
# Evaluation is carried out with a batch size of 1
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

In [ ]:
# Chech the errors made by the model
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))